In [ ]:
from simple_salesforce import Salesforce as sf
login_sf = sf(username = 'svc_rao@groupon.com', password = 'Password-1Je8IR8frn0gkbS68Eq4OHZmM', security_token = '')

import pandas as pd
from simple_salesforce import bulk
from megatron import saving as m
import pygsheets
import os


In [ ]:
### Account IDs as input to process
account_ids =  ['001C0000017HbQVIA0','001C000001gCqa3IAC','0013c00001oSlw7AAC','0013c00001paVFVAA2','001C000001cqaIKIAY','001C000001l8hmVIAQ','0013c00001xfss9AAA','0013c00001xgGYrAAM','0013c00001nlblgAAA','0013c00001pYGZlAAO','001C0000017HeK5IAK','0013c00001nlw72AAA','001C000001FlEnzIAF','0013c00001xfPnMAAU','0013c00001tcykYAAQ','0013c00001tcij5AAA','0013c00001qybE0AAI','0013c00001qybG6AAI','0013c00001zGCbAAAW','0013c00001qz91oAAA','0013c00001qybLBAAY','0013c00001qybMJAAY','0013c00001tcx8TAAQ','0013c00001zGCWUAA4','0013c00001tdokuAAA','0013c00001qybPcAAI','0013c00001qyf74AAA','0013c00001td6FiAAI','0013c00001zGKIKAA4','0013c00001nlRleAAE','0013c00001qyyInAAI','001C000001TnND0IAN','0013c00001zFWLZAA4','0013c00001pYTksAAG','0013c00001tcIhaAAE','0013c00001te4V5AAI','001C000001koDMuIAM','001C0000017HKuRIAW','001C000001nBw0SIAS','001C000001DPDydIAH','0013c00001pa9heAAA','001C0000017jCP7IAM','001C0000017HA8JIAW','001C000001eL0EjIAK','0013c00001oTc6XAAS','0013c00001pZayeAAC','001C000001ePzPnIAK','001C000001hJnHtIAK','001C000001RsukmIAB','001C0000017Hcv6IAC','0013c00001qy9ViAAI','0013c00001nExaxAAC','001C000001nHRJNIA4','0013c00001pbDhzAAE','0013c00001ny98uAAA','0013c00001sKI19AAG','001C0000017HQKPIA4','001C0000017Hc3NIAS','001C000001nzNesIAE','0013c00001qz7sqAAA','0013c00001nOYrWAAW','0013c00001pbNxpAAE','0013c00001xgRDzAAM','0013c00001pYlW2AAK','001C0000017HCOJIA4','001C0000017HZZPIA4','0013c00001qzULsAAM','001C000001Qn6YxIAJ','001C000001cLgTfIAK','001C000001BQge8IAD','001C000001CVIhYIAX','001C000001aSFbzIAG','0013c00001tdrlRAAQ','0013c00001pZCH2AAO','0013c00001nEhDGAA0','001C000001igeboIAA','0013c00001qzZ4kAAE','001C000001mslYJIAY','0013c00001nlMavAAE','001C000001nCH1aIAG','0013c00001qym77AAA','0013c00001tcGK1AAM','001C0000017HAJFIA4','001C000001nCO9LIAW','001C000001VwbQeIAJ','001C000001YgazVIAR','0013c00001sK7pwAAC','0013c00001sKS7EAAW','001C0000017Hd0yIAC','001C0000017HF3zIAG','001C0000017HX2tIAG','0013c00001xfzUsAAI','001C000001lsp1xIAA','001C000001FlF3NIAV','001C000001aViDnIAK','001C0000017HM1qIAG','0013c00001oT8dgAAC','001C0000017HaT2IAK','001C000001hMBQXIA4','001C000001QozhBIAR','001C0000017HJpfIAG','001C000001PZ1MHIA1','0013c00001uue6AAAQ','0013c00001tdt5RAAQ','0013c00001pXs6ZAAS','0013c00001xfst7AAA','0013c00001r0gU6AAI','0013c00001nlzhVAAQ','001C000001j2R29IAE','001C000001RvbtKIAR','001C000001TmteUIAR','0013c00001nlR7oAAE','001C0000017HfAzIAK','0013c00001oT5BfAAK','001C000001cpNS7IAM','0013c00001qyWioAAE','001C000001jUtuwIAC','0013c00001teCPYAA2','0013c00001qyIJRAA2','0013c00001pbkkLAAQ','001C000001gFKquIAG','0013c00001oTaCSAA0','001C000001lUWFbIAO','001C000001ZZq4FIAT','001C000001dDkhaIAC','001C000001lUW07IAG','0013c00001pZwasAAC','001C0000017HEHUIA4','0013c00001ugSz4AAE','001C000001YKhv8IAD','001C000001UneCnIAJ']

account_list = "','".join(account_ids)

In [ ]:
### get DUNS numbers per account per possible in order to query the D&B Optimzer object
output = login_sf.bulk.Account.query(f"SELECT Id,DNBoptimizer__DnBCompanyRecord__c FROM Account WHERE Id in ('{account_list}')")

df = pd.DataFrame(output)
# remove first column with attribute data
df = df.iloc[:,1:]
# filter out datapoints without DB_Ids numbers
db_map = df.dropna()

# prepeare list of DB_Ids to be queried
db_ids = db_map['DNBoptimizer__DnBCompanyRecord__c'].tolist()
db_id_list = "','".join(db_ids)


In [1]:
### get D&B Optimizer data for the DUNS numbers found
output = login_sf.bulk.DNBoptimizer__DnBCompanyRecord__c.query(f"SELECT Id,DNBoptimizer__BusinessName__c,DNBoptimizer__DUNSNumber__c,DNBoptimizer__GlobalUltimateDUNSNumber__c,DNBoptimizer__DelistedIndicator__c,DNBoptimizer__EmployeeCountTotal__c,DNBoptimizer__LocationType__c,DNBoptimizer__NAICSDescription1__c,DNBoptimizer__NAICSDescription2__c,DNBoptimizer__NAICSDescription3__c,DNBoptimizer__OutofBusinessIndicator__c,DNBoptimizer__PrimaryCityName__c,DNBoptimizer__PrimaryCountryCode_ISO__c,DNBoptimizer__PrimaryCountryName__c,DNBoptimizer__PrimaryPostalCode__c,DNBoptimizer__PrimaryStreetAddress__c,DNBoptimizer__TelephoneNumber__c,DNBoptimizer__WebAddress__c,DNBoptimizer__YearStarted__c FROM DNBoptimizer__DnBCompanyRecord__c where Id in ('{db_id_list}')")

#prepare table from sf_output
df = pd.DataFrame(output)
db_data = df.iloc[:,1:]


NameError: name 'login_sf' is not defined

In [ ]:
### Join datasets
# rename columns in order to join
db_map.columns=['AccountId','Id']

df = db_map.join(db_data.set_index('Id'), on='Id')


In [ ]:
### push data to google-sheet
m.df_to_gdoc(df,'1wHvQnt1eIs0naWO2fdLglCY0NQjON_4OlQzdGukZj-8','db_data')

print("Success")
